# To visualize your training result
we use `hiplot` to visualize your training result in a jupyter notebook 

In [1]:
!pip install hiplot

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
import sys
import time
import numpy as np
import pickle as pkl
import hiplot as hip

In [3]:
from litebo.optimizer.generic_smbo import SMBO
from test.reproduction.so.so_benchmark_function import get_problem
from test.reproduction.test_utils import timeit, seeds

/opt/anaconda3/envs/lite_bo/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [4]:
problem_str = 'branin'
max_runs = 10
initial_runs = 3
init_strategy = 'random_explore_first'
surrogate_type = 'gp'
optimizer_str = 'scipy'
if optimizer_str == 'scipy':
    acq_optimizer_type = 'random_scipy'
elif optimizer_str == 'local':
    acq_optimizer_type = 'local_random'
else:
    raise ValueError('Unknown optimizer %s' % args.optimizer)
rep = 1
start_id = 0
mth = 'litebo'

problem = get_problem(problem_str)
cs = problem.get_configspace(optimizer='smac')
time_limit_per_trial = 600
task_id = '%s_%s' % (mth, problem_str)

# Train and get the config_list and perf_list

In [5]:
def evaluate(mth, run_i, seed):
    print(mth, run_i, seed, '===== start =====', flush=True)

    def objective_function(config):
        y = problem.evaluate_config(config)
        res = dict()
        res['config'] = config
        res['objs'] = (y,)
        res['constraints'] = None
        return res

    bo = SMBO(objective_function, cs,
              surrogate_type=surrogate_type,            # default: gp
              acq_optimizer_type=acq_optimizer_type,    # default: random_scipy
              initial_runs=initial_runs,                # default: 3
              init_strategy=init_strategy,              # default: random_explore_first
              max_runs=max_runs,
              time_limit_per_trial=time_limit_per_trial, task_id=task_id, random_state=seed)
    # bo.run()
    config_list = []
    perf_list = []
    time_list = []
    global_start_time = time.time()
    for i in range(max_runs):
        config, trial_state, objs, trial_info = bo.iterate()
        global_time = time.time() - global_start_time
        print(seed, i, objs, config, trial_state, trial_info, 'time=', global_time)
        config_list.append(config)
        perf_list.append(objs[0])
        time_list.append(global_time)
    
    try:
        bo.visualize_jupyter()
    except:
        raise NotImplementedError('bo.visualize_jupyter() meet some problems')
    
    return config_list, perf_list, time_list


with timeit('%s all' % (mth,)):
    for run_i in range(start_id, start_id + rep):
        seed = seeds[run_i]
        with timeit('%s %d %d' % (mth, run_i, seed)):
            # Evaluate
            config_list, perf_list, time_list = evaluate(mth, run_i, seed)

            # Save result
            print('=' * 20)
            print(seed, mth, config_list, perf_list, time_list)
            print(seed, mth, 'best perf', np.min(perf_list))

            timestamp = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(time.time()))
            dir_path = 'logs/so_benchmark_%s_%d/%s/' % (problem_str, max_runs, mth)
            file = 'benchmark_%s_%04d_%s.pkl' % (mth, seed, timestamp)
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
            with open(os.path.join(dir_path, file), 'wb') as f:
                save_item = (config_list, perf_list, time_list)
                pkl.dump(save_item, f)
            print(dir_path, file, 'saved!', flush=True)

[litebo all]Start.
[litebo 0 4774]Start.
litebo 0 4774 ===== start =====
[INFO] [2021-03-17 22:54:07,055:Lite-BO-litebo_branin-[2021-03-17 22:54:07]] In the 1-th iteration, the objective value: (24.129964413622268,)
4774 0 (24.129964413622268,) Configuration:
  x1, Value: 2.5
  x2, Value: 7.5
 0 None time= 0.022910118103027344
[INFO] [2021-03-17 22:54:07,079:Lite-BO-litebo_branin-[2021-03-17 22:54:07]] In the 2-th iteration, the objective value: (266.87545772568916,)
4774 1 (266.87545772568916,) Configuration:
  x1, Value: -4.650963559365345
  x2, Value: 0.15097536559954217
 0 None time= 0.046996116638183594
[INFO] [2021-03-17 22:54:07,104:Lite-BO-litebo_branin-[2021-03-17 22:54:07]] In the 3-th iteration, the objective value: (112.99109683630043,)
4774 2 (112.99109683630043,) Configuration:
  x1, Value: 9.868015819326814
  x2, Value: 13.441442125698543
 0 None time= 0.07118701934814453
[INFO] [2021-03-17 22:54:07,449:Lite-BO-litebo_branin-[2021-03-17 22:54:07]] In the 4-th iteration, 

<IPython.core.display.Javascript object>

4774 litebo [Configuration:
  x1, Value: 2.5
  x2, Value: 7.5
, Configuration:
  x1, Value: -4.650963559365345
  x2, Value: 0.15097536559954217
, Configuration:
  x1, Value: 9.868015819326814
  x2, Value: 13.441442125698543
, Configuration:
  x1, Value: 4.53859534106417
  x2, Value: 7.72710041549182
, Configuration:
  x1, Value: -2.9470342311735314
  x2, Value: 6.895503183087984
, Configuration:
  x1, Value: 7.340308001353556
  x2, Value: 6.708248457423043
, Configuration:
  x1, Value: 1.1332381219321883
  x2, Value: 6.774051094740018
, Configuration:
  x1, Value: -5.0
  x2, Value: 8.404597437398893
, Configuration:
  x1, Value: 0.8299045316135079
  x2, Value: 7.1509790911646025
, Configuration:
  x1, Value: -5.0
  x2, Value: 5.627158932919043
] [24.129964413622268, 266.87545772568916, 112.99109683630043, 47.896748319314426, 24.754131850440363, 44.20573285803001, 19.88524657840387, 89.86067243644428, 22.158828527258798, 146.36200226791502] [0.022910118103027344, 0.046996116638183594, 0

In [6]:
bo

NameError: name 'bo' is not defined

In [ ]:
config_list

In [ ]:
config_list[0]

In [ ]:
type(config_list[0])

In [ ]:
config_list[0].get_dictionary()

In [ ]:
perf_list[0]

# prepare the visualize_data

In [ ]:
visualize_data = []
for config in config_list:
    visualize_data.append(config.get_dictionary())
for tmp1, tmp2 in zip(visualize_data, perf_list):
    tmp1['perf'] = tmp2

In [ ]:
visualize_data

In [ ]:
hip.Experiment.from_iterable(visualize_data).display()